In [1]:
!pip install nltk rouge-score bert-score scikit-learn pillow gradio

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.7 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=562947c9fdbc65b6dfe6c24cb18abe5e27653903a5854f5619a6981c5f7f3d7b
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge-score


In [8]:


import gradio as gr
import numpy as np
from PIL import Image
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
from bert_score import score as bert_score
from scipy.stats import ttest_rel
import gradio as gr
import numpy as np
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
from bert_score import score as bert_score
from scipy.stats import ttest_rel
from PIL import Image

In [12]:


# -------------------------------
# Metrics computation
# -------------------------------
rouge = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

def compute_metrics(preds, golds):
    acc = np.mean([p.strip().lower() == g.strip().lower() for p, g in zip(preds, golds)])
    bleu = np.mean([sentence_bleu([g.split()], p.split()) for p, g in zip(preds, golds)])
    rouge1 = np.mean([rouge.score(g, p)["rouge1"].fmeasure for p, g in zip(preds, golds)])
    rougel = np.mean([rouge.score(g, p)["rougeL"].fmeasure for p, g in zip(preds, golds)])
    P, R, F = bert_score(preds, golds, lang="en", verbose=False)
    return {
        "accuracy": acc,
        "bleu": bleu,
        "rouge1": rouge1,
        "rougel": rougel,
        "bert_f1": float(F.mean())
    }

# -------------------------------
# Evaluation function
# -------------------------------
def evaluate_answers(chatgpt_answers, gemini_answers, finetuned_answers, gold_answers):
    chatgpt_list = [x.strip() for x in chatgpt_answers.strip().split("\n") if x.strip()]
    gemini_list = [x.strip() for x in gemini_answers.strip().split("\n") if x.strip()]
    finetuned_list = [x.strip() for x in finetuned_answers.strip().split("\n") if x.strip()]
    gold_list = [x.strip() for x in gold_answers.strip().split("\n") if x.strip()]

    n = len(gold_list)
    if not (len(chatgpt_list) == len(gemini_list) == len(finetuned_list) == n):
        return "Error: Number of answers and golds must match for all models."

    results = {
        "ChatGPT": compute_metrics(chatgpt_list, gold_list),
        "Gemini": compute_metrics(gemini_list, gold_list),
        "Finetuned": compute_metrics(finetuned_list, gold_list)
    }

    return results


In [13]:

# -------------------------------
# Gradio Interface
# -------------------------------
with gr.Blocks() as demo:
    gr.Markdown("## Multi-Model Answer Evaluation 🧪")

    with gr.Row():
        with gr.Column():
            image_input = gr.Image(type="pil", label="Upload Medical Image")
            question_input = gr.Textbox(lines=2, placeholder="Type your question here", label="Question")
            chatgpt_input = gr.Textbox(lines=5, placeholder="Paste ChatGPT answers (one per line)", label="ChatGPT Answers")
            gemini_input = gr.Textbox(lines=5, placeholder="Paste Gemini answers (one per line)", label="Gemini Answers")
            finetuned_input = gr.Textbox(lines=5, placeholder="Paste Finetuned answers (one per line)", label="Finetuned Answers")
            gold_input = gr.Textbox(lines=5, placeholder="Paste Gold answers (one per line)", label="Gold Answers")
            eval_btn = gr.Button("Compute Metrics")

        with gr.Column():
            metrics_output = gr.JSON(label="Metrics per Model")

    eval_btn.click(
        evaluate_answers,
        inputs=[chatgpt_input, gemini_input, finetuned_input, gold_input],
        outputs=metrics_output
    )

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://65fa78e8e89365cea0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
